# Making a figure looking at $b^2$

This notebook is used to make a figure for the halo bais paper. It will use some modules that aren't dependencies of this one, so it likely won't run for a user.

In [1]:
import numpy as np
import bias_emulator
import hmf_emulator
import aemulus_data as AD
import aemulus_extras as ae
import matplotlib.pyplot as plt
from scipy.interpolate import InterpolatedUnivariateSpline as IUS
%matplotlib inline

In [2]:
plt.rc("font", size=16, family="serif")
plt.rc("errorbar", capsize=3)
plt.rc("text", usetex=True)

In [10]:
cosmo_index = 0
#Note: we remove sigma8 from the list of parameters
cosmology = AD.test_box_cosmologies()[cosmo_index][:-1]
cosmo_dict = {"omega_b":cosmology[0], "omega_cdm":cosmology[1], "w0":cosmology[2],
             "n_s":cosmology[3], "ln10As":cosmology[4],"H0":cosmology[5],"N_eff":cosmology[6]}
print("Cosmology for testing simulation %d loaded"%cosmo_index)
Extras = ae.Extras(cosmo_index, testing=True) #Testing cosmos == MedBox cosmos

Cosmology for testing simulation 0 loaded


In [11]:
#Create emulators
bias_emu = bias_emulator.bias_emulator()
bias_emu.set_cosmology(cosmo_dict)
hmf_emu = hmf_emulator.hmf_emulator()
hmf_emu.set_cosmology(cosmo_dict)

In [13]:
#Make a function to get halo data
def get_halo_data(snap):
    _, Mlos, Mhis, Masses = np.loadtxt(datapath+"MedBox000_Z%d_halo_means_hmcf.txt"%snap, 
                                       unpack=True)
    Mhis[-1] = 10**16.5
    return Mlos, Mhis, Masses

redshifts = 1./AD.scale_factors() - 1

In [14]:
#Write a function to get the correct bin-averaged bias
def bin_ave_biases_in_snap(snap):
    
    Mlos, Mhis, Masses = get_halo_data(snap)
    
    Marr = np.logspace(np.log10(Mlos[0]), np.log10(Mhis[-1]), 1000)
    lnMarr = np.log(Marr)
    bias_at_Marr = bias_emu.bias(Marr, redshifts[snap])
    dndM_at_Marr = hmf_emu.dndM(Marr, redshift[snap])
    
    bhmf_spl = IUS(lnMarr, bias_at_Marr*dndM_at_Marr*Marr)
    hmf_spl = IUS(lnMarr, dndM_at_Marr*Marr)

    lnMlos = np.log(Mlos)
    lnMhis = np.log(Mhis)
    
    output = np.zeros_like(Mlos)
    for i in range(len(output)):
        num = bhmf_spl.integral(lnMlos[i], lnMhis[i])
        den = hmf_spl.integral(lnMlos[i], lnMhis[i])
        output[i] = num/den
    return output

In [15]:
#Write a function to get the correct bin-averaged bias^2
def bin_ave_b2_in_snap(snap):
    
    Mlos, Mhis, Masses = get_halo_data(snap)
    
    Marr = np.logspace(np.log10(Mlos[0]), np.log10(Mhis[-1]), 1000)
    lnMarr = np.log(Marr)
    bias_at_Marr = bias_emu.bias(Marr, redshifts[snap])
    dndM_at_Marr = hmf_emu.dndM(Marr, redshift[snap])
    
    b2hmf_spl = IUS(lnMarr, bias_at_Marr*bias_at_Marr*dndM_at_Marr*Marr)
    hmf_spl = IUS(lnMarr, dndM_at_Marr*Marr)

    lnMlos = np.log(Mlos)
    lnMhis = np.log(Mhis)
    
    output = np.zeros_like(Mlos)
    for i in range(len(output)):
        num = b2hmf_spl.integral(lnMlos[i], lnMhis[i])
        den = hmf_spl.integral(lnMlos[i], lnMhis[i])
        output[i] = num/den
    return output